In [1]:
import pickle

import pandas as pd
import numpy as np
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.palettes
import bokeh.plotting
import bokeh.transform
import bokeh.embed
import json

import sp_project.app_state
from sp_project.notebook_tools import default_clients
import sp_project.data_collection.openweather_prediction as weatherprediction
from sp_project.data_preparation.create_model import *
from sp_project.data_preparation.prediction_preparation import *

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
lon = 7.70
lat = 46.42

In [3]:
with open("../../src/sp_project/backend_server/assets/prediction-model.pickle", "rb") as fh:
    model = pickle.load(fh)

In [4]:
async with default_clients() as (OWclient, db):
    app_state = sp_project.app_state.AppState(ow_client=OWclient, db_client=db, model=model)
    result = await extract_predictions_daily(app_state, lon=lon, lat=lat)
    
    features = prepare_prediction_features(result, lat)
    prediction = energy_prediction(app_state.model, features)

In [5]:
prediction.head()

Energy Features,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
dt,,,,,,,
2023-07-25 11:00:00+00:00,206.904695,5558.692905,504.105130,7651.080812,3579.730808,5237.760222,22738.274572
2023-07-26 11:00:00+00:00,16.121685,380.298117,2008.479357,1334.326223,292.387164,804.193107,4835.805653
2023-07-27 11:00:00+00:00,10.961515,277.207137,1966.302637,1190.827167,221.016880,701.447996,4367.763331
2023-07-28 11:00:00+00:00,92.959534,2919.467342,385.961400,4213.966617,1831.595728,2800.472574,12244.423195
2023-07-29 11:00:00+00:00,74.845877,2197.838106,1044.252362,3444.088075,1410.354885,2272.722352,10444.101657


In [6]:
prediction_source = bokeh.models.ColumnDataSource(prediction)

column_names = "nuclear solar wind water_reservoir water_pump water_river".split()
colors = "#D55E00 #F0E442 #BBBBBB #009E73 #0072B2 #56B4E9".split()

middle = len(prediction) // 2
selection_range = 2

# Selected-Part-Plot
p = bokeh.plotting.figure(
    height=300,
    width=800,
    x_axis_type="datetime",
    x_axis_location="above",
    x_range=(prediction.index[middle - selection_range], prediction.index[middle + selection_range]),
)

p.yaxis.axis_label = 'Energy Production [MW]'

p.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
    legend_label=column_names,
)

# Overview-Plot
overview = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130,
    width=800,
    x_axis_type="datetime",
    y_range=p.y_range,
    y_axis_type=None,
    tools="hover",
    toolbar_location=None,
)

overview.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
)

p.legend.location = "top_left"

range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

overview.ygrid.grid_line_color = None
overview.add_tools(range_tool)

overview_fig = bokeh.layouts.column(p, overview)

bokeh.plotting.show(overview_fig)

In [15]:
x = round(prediction.iloc[:, :-1].sum(), 2)

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'Energy Features'})
data['angle'] = data['value'] / data['value'].sum() * 2 * np.pi
data['color'] = "#BBBBBB #F0E442 #D55E00 #009E73 #0072B2 #56B4E9".split()

p = bokeh.plotting.figure(
    height=350,
    title="Average Production for the next 8 Days",
    toolbar_location=None,
    tools="hover",
    tooltips="@country: @value",
    x_range=(-0.5, 1.0)
)

p.wedge(x=0, y=1, radius=0.4,
        start_angle=bokeh.transform.cumsum('angle', include_zero=True), end_angle=bokeh.transform.cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Energy Features', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

pie_fig = bokeh.layouts.column(p)

bokeh.plotting.show(pie_fig)

### Save the Graphics as JSON-Files

In [16]:
with open("energy_prediction_overview.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(overview_fig), fh)

In [17]:
with open("energy_prediction_pieplot.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(pie_fig), fh)

## Jupyter-Notebook Footer-Info

In [18]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 22.5.0
Datetime: 2023-07-25 09:50:11
Python Version: 3.11.4
-----------------------------------
